In [10]:
import json
from collections import defaultdict
from tqdm import tqdm

# 파일 경로
input_file = "merged_dataset.json"
output_file = "merged_dataset_5up_users_only.json"

# 1단계: 사용자 리뷰 수 세기
user_review_counts = defaultdict(int)

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        user_id = obj.get("user_id")
        if user_id:
            user_review_counts[user_id] += 1

# 2단계: 리뷰 수 5개 이상 유저만 필터링
qualified_users = {uid for uid, count in user_review_counts.items() if count >= 5}
print(f"🧾 리뷰 수 5개 이상 유저 수: {len(qualified_users)}")

filtered_reviews = []

with open(input_file, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="🔍 리뷰 필터링 중"):
        obj = json.loads(line)
        if obj.get("user_id") in qualified_users:
            filtered_reviews.append(obj)

# 저장
with open(output_file, "w", encoding="utf-8") as f:
    for obj in filtered_reviews:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print(f"✅ 필터링 완료: {len(filtered_reviews)}개 리뷰 저장 → {output_file}")


🧾 리뷰 수 5개 이상 유저 수: 28465


🔍 리뷰 필터링 중: 451185it [00:06, 70896.04it/s]


✅ 필터링 완료: 451185개 리뷰 저장 → merged_dataset_5up_users_only.json


#### 감성분석 결과에 유저ID, 식당ID ,평점 추가

In [9]:
import json
from tqdm import tqdm

# 파일 경로
full_data_file = "merged_dataset.json"
target_file = "review_5up_5aspect_3sentiment.json"
output_file = "review_5up_5aspect_3sentiment_with_ids.json"

# 전체 리뷰 데이터에서 review_id → 필요한 정보 맵 생성
id_map = {}

with open(full_data_file, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        rid = obj.get("review_id")
        if rid:
            id_map[rid] = {
                "user_id": obj.get("user_id"),
                "business_id": obj.get("business_id"),
                "stars": obj.get("review_stars"),  # 별점 키명 일치
                "review_useful": obj.get("review_useful"),
                "review_date": obj.get("review_date")
            }

print(f" 전체 리뷰에서 매핑된 review_id 수: {len(id_map)}")

# 대상 파일 읽고 정보 추가
updated = []
missing = []

with open(target_file, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="🔄 ID 및 평점 추가 중"):
        obj = json.loads(line)
        rid = obj.get("review_id")
        if rid in id_map:
            obj.update(id_map[rid])  # 여러 필드 추가
            updated.append(obj)
        else:
            missing.append(obj)  # 매칭 안 되는 경우 따로 보관

# 결과 저장
with open(output_file, "w", encoding="utf-8") as f:
    for obj in updated:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print(f" 저장 완료: 총 {len(updated)}건 → {output_file}")
if missing:
    print(f" 매칭 안된 리뷰 수: {len(missing)}")


 전체 리뷰에서 매핑된 review_id 수: 451185


🔄 ID 및 평점 추가 중: 452505it [00:23, 19476.89it/s]


 저장 완료: 총 451185건 → review_5up_5aspect_3sentiment_with_ids.json
 매칭 안된 리뷰 수: 1320


#### 추천형식에 맞게 변환

In [ ]:
import json

input_file = "review_5up_5aspect_3sentiment_with_ids.json"
output_file = "review_5up_5aspect_3sentiment_vectorized_clean.json"

def sentiment_to_vector(sentiment_dict):
    aspects = ["food", "service", "price", "ambience", "location"]
    polarities = ["Negative", "Neutral", "Positive"]
    vector = []
    for asp in aspects:
        scores = sentiment_dict.get(asp, {}).get("scores", {})
        for pol in polarities:
            vector.append(scores.get(pol, 0.0))
    return vector

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    
    for line in fin:
        obj = json.loads(line)
        
        # 벡터 생성
        sentiment_vec = sentiment_to_vector(obj.get("sentiment", {}))
        
        # 필요한 필드만 추출 및 재구성
        cleaned = {
            "review_id": obj.get("review_id"),
            "user_id": obj.get("user_id"),
            "business_id": obj.get("business_id"),
            "stars": obj.get("stars"),
            "review_date": obj.get("review_date"),
            "sentiment_vector": sentiment_vec
        }
        
        fout.write(json.dumps(cleaned, ensure_ascii=False) + "\n")

print(f"✅ 완료: text와 sentiment 제거 후 저장 → {output_file}")


✅ 완료: text와 sentiment 제거 후 저장 → review_5up_5aspect_3sentiment_vectorized_clean.json


#### 유저별 평균 리뷰수 확인

In [7]:
import json
from collections import defaultdict

# 파일 경로
INPUT_FILE = "review_5up_5aspect_3sentiment_vectorized_clean.json"

# 유저별 리뷰 수 저장
user_review_counts = defaultdict(int)

# 데이터 로딩
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        review = json.loads(line)
        user_id = review["user_id"]
        user_review_counts[user_id] += 1

# 통계 계산
total_users = len(user_review_counts)
total_reviews = sum(user_review_counts.values())
avg_reviews_per_user = total_reviews / total_users

print(f"📊 총 유저 수: {total_users}")
print(f"📝 총 리뷰 수: {total_reviews}")
print(f"📈 유저당 평균 리뷰 수: {avg_reviews_per_user:.2f}")


📊 총 유저 수: 28518
📝 총 리뷰 수: 452505
📈 유저당 평균 리뷰 수: 15.87


#### 훈련 80% / 평가20% 로 분할

In [5]:
import json
import math
from collections import defaultdict
from tqdm import tqdm

# 설정
INPUT_FILE = "review_5up_5aspect_3sentiment_vectorized_clean.json"
TRAIN_FILE = "train_80.json"
TEST_FILE = "test_20.json"
MIN_REVIEWS = 5

# 유저별 리뷰 모으기
user_reviews = defaultdict(list)

with open(INPUT_FILE, encoding="utf-8") as f:
    for line in f:
        review = json.loads(line)
        user_reviews[review["user_id"]].append(review)

# 1차 분할
train_data, raw_test_data = [], []

for uid, reviews in tqdm(user_reviews.items(), desc="20% 비율로 분할 중"):
    if len(reviews) < MIN_REVIEWS:
        continue

    sorted_reviews = sorted(reviews, key=lambda x: x.get("date", x.get("review_id")))
    k = max(1, math.ceil(len(reviews) * 0.2))

    train_data.extend(sorted_reviews[:-k])
    raw_test_data.extend(sorted_reviews[-k:])

# Train에서 등장한 유저와 business ID 수집
train_user_ids = set(r["user_id"] for r in train_data)
train_business_ids = set(r["business_id"] for r in train_data)

# 평가셋: 유저와 비즈니스 모두 학습셋에 등장한 경우만 유지
test_data = [
    r for r in raw_test_data
    if r["user_id"] in train_user_ids and r["business_id"] in train_business_ids
]

# 저장
with open(TRAIN_FILE, "w", encoding="utf-8") as f:
    for r in train_data:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(TEST_FILE, "w", encoding="utf-8") as f:
    for r in test_data:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✅ 훈련 데이터 개수: {len(train_data)}")
print(f"✅ 평가 데이터 개수 (유저+비즈니스 교집합 유지): {len(test_data)}")
print(f"📊 사용자 수 (총): {len(user_reviews)}")


20% 비율로 분할 중: 100%|██████████| 28465/28465 [00:00<00:00, 124480.24it/s]


✅ 훈련 데이터 개수: 350032
✅ 평가 데이터 개수 (유저+비즈니스 교집합 유지): 101116
📊 사용자 수 (총): 28465


In [4]:
import json

def count_unique_users(file_path):
    user_set = set()
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            review = json.loads(line)
            user_set.add(review["user_id"])
    return len(user_set)

train_file = "train_80.json"
test_file = "test_20.json"

train_user_count = count_unique_users(train_file)
test_user_count = count_unique_users(test_file)

print(f"👥 훈련 데이터의 유저 수: {train_user_count}")
print(f"🧪 평가 데이터의 유저 수: {test_user_count}")


👥 훈련 데이터의 유저 수: 28465
🧪 평가 데이터의 유저 수: 28464
